In [2]:
# Import Libraries
import numpy as np
import pandas as pd
import ast 
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import os



# Preprocessing


In [3]:
import ruptures as rpt

def trajectory_cruise_segmentation(track, feasibility):

        # Load your data
        data = track
        matching = True

        # Define the signals that you are interested in
        speed = data['speed_ft_s'].values
        altitude = data['alt_ft_agl'].values

        # Standardize the variables to have zero mean and unit variance
        epsilon = 1e-8
        speed = (speed - np.mean(speed)) / (np.std(speed) + epsilon)  #Epsilon helps prevent division by zero cases

        #speed = (speed - np.mean(speed)) / np.std(speed)
        altitude = (altitude - np.mean(altitude)) / np.std(altitude)

        # Create a composite signal
        composite_signal = np.sqrt(speed**2 + altitude**2)

        # Define algorithm configuration
        algo = rpt.Pelt(model="rbf").fit(composite_signal)

        # Detection
        result = algo.predict(pen=3)  # You might need to adjust the penalty value
        if len(feasibility) != len(result):
           #print(f"Length mismatch: Feasibility length is {len(feasibility)}, but trajectory length is {len(result)}.")
           matching = False


        # Create a new column 'segment' and set it initially to 0
        data['segment'] = 0
        boundaries = [(0, result[0])]
        data.loc[0:result[0], 'segment'] = 0
        
        # Loop over the result to assign each segment a unique ID
        for i, val in enumerate(result[:-1]):            
            if i == len(result)-2:
                boundaries.append((val, result[i+1]-1))
            else:
                boundaries.append((val, result[i+1]))
            data.loc[val:result[i+1], 'segment'] = i+1
            

        return matching, data
        # Save the segmented data back to CSV
        # data.to_csv('your_data_file_with_segments.csv', index=False)
   

In [4]:
def preprocess_file(file_path, scaler, encoder):
    # Load data
    data = pd.read_csv(file_path)
    trajectory_data = data['flight_dynamics'].apply(lambda x: pd.read_json(x))

    trajectory_data[0].drop(['lat_deg', 'down_ft', 'lon_deg', 'theta_rad', 'psi_rad', 'phi_rad', 'segment'], axis=1, inplace=True)

    boundaries, trajectory = trajectory_cruise_segmentation(trajectory_data[0])
    segment_features = trajectory.groupby('segment').apply(lambda x: process_segment(x, False)).reset_index(drop=True)

    #trajectory_data = trajectory_data[0].merge(segment_features, on='segment', how='left')

    # Apply Feature Engineering
    # Broadcast each feature to all rows within the same segment
    for col in segment_features.columns:
        trajectory_data[0][col] = np.nan
        trajectory_data[0][col] = trajectory['segment'].map(segment_features[col])

    
    scaled_data = scaler.fit_transform(trajectory_data[0])

    trajectory_scaled = pd.DataFrame(scaled_data)          ########### Removing scaling #############

    vehicle_type = data['vehicle_type'][0]

    # Feasibility
    feasibility = data['feasibility'][0]
    feasibility = ast.literal_eval(feasibility)
    feasibility_temp = []
    for (i, seg_feasibility) in enumerate(feasibility):
        segment_size = len(trajectory_data[0][trajectory_data[0].segment==i])
        feasibility_temp = feasibility_temp + [seg_feasibility] * segment_size
        
        
    feasibility = feasibility_temp

    return trajectory_scaled, vehicle_type, feasibility, 241

def preprocess_file_per_segment(file_path, scaler, encoder):
    # Load data
    data = pd.read_csv(file_path)
    
    # Feasibility
    feasibility = data['feasibility'][0]
    feasibility = ast.literal_eval(feasibility)

    #Trajectory
    trajectory_data = data['flight_dynamics'].apply(lambda x: pd.read_json(x))

    trajectory_data[0].drop(['segment'], axis=1, inplace=True)

    matching, trajectory = trajectory_cruise_segmentation(trajectory_data[0], feasibility)
    
    segment_count = trajectory['segment'].nunique()  # Count unique segments
    
    # Apply Feature Engineering
    engineered_trajectory = trajectory.groupby('segment').apply(lambda x: process_segment(x, True)).reset_index(drop=True)

    vehicle_type = data['vehicle_type'][0]

    return engineered_trajectory, vehicle_type, feasibility, segment_count, matching

def process_segment(group, segmented):
    # Ensure the group is sorted by Time
    group = group.sort_values(by='Time')

    # Calculate time difference
    time_elapsed = group['Time'].iloc[-1] - group['Time'].iloc[0]
    
    airspeed_value = group['speed_ft_s'].mean()
    # Calculate distance
    distance = airspeed_value * time_elapsed

    # Calculate altitude

    altitude = group['alt_ft_msl'].mean() #(group['alt_ft_msl'].iloc[-1] + group['alt_ft_msl'].iloc[0])/2
    
    if segmented:

        return pd.Series({'distance': distance, 'airspeed': airspeed_value, 'altitude': altitude})
    else:
        return pd.Series({'distance': distance, 'airspeed_const': airspeed_value, 'elapsed_time': time_elapsed})



In [5]:
from collections import defaultdict

data_by_vehicle_type = {}

# Assuming you know the range of values in your trajectory data and the different vehicle types
scaler = StandardScaler() #feature_range=(0, 1)) # Adjust the range according to your data
encoder = OneHotEncoder(sparse= False)

current_dir = os.getcwd() #os.path.dirname(os.path.abspath())
feasibility_file_path = os.path.join(os.path.dirname(current_dir), 'logs', 'feasibility_data')

file_paths = [os.path.join(feasibility_file_path, file) for file in os.listdir(feasibility_file_path)]
all_trajectories = []
all_vehicle_types = []
all_feasbilities = []
segment_counts = {}

for file_path in file_paths:
    trajectory, vehicle_type, feasibility, segment_count, traj_feas_match = preprocess_file_per_segment(file_path, scaler, encoder)
    segment_counts[file_path] = segment_count

    if vehicle_type not in data_by_vehicle_type:
        data_by_vehicle_type[vehicle_type] = []
    if traj_feas_match:
        data_by_vehicle_type[vehicle_type].append((trajectory, feasibility))
        all_trajectories.append(trajectory)
        all_vehicle_types.append(vehicle_type)
        all_feasbilities.append(feasibility)

file_with_max_segments = max(segment_counts, key=segment_counts.get)
max_count = segment_counts[file_with_max_segments]


In [6]:


def pad_trajectory(trajectory, max_length=max_count, padding_value=0.0):
    # Calculate the number of time steps to pad
    padding_length = max_length - trajectory.shape[0]

    # Create the padding array
    padding = np.full((padding_length, trajectory.shape[1]), padding_value, dtype='float32')

    # Concatenate the trajectory and the padding
    padded_trajectory = np.concatenate([trajectory, padding], axis=0)
    if padded_trajectory.shape[0] != 241:
        print('Not padded correctly ', padded_trajectory.shape[0])

    return padded_trajectory

def pad_feasibility(feasibility, max_length= max_count, padding_value = False):
    
    padding_length = max_length - len(feasibility)
    padding = np.full((padding_length), padding_value)
    padded_feasibility = np.concatenate([feasibility, padding], axis=0)

    return padded_feasibility

    
def create_sequences(trajectories, feasibilities, sequence_length):
    X, y = [], []
    for trajectory, feasibility in zip(trajectories, feasibilities):
        for i in range(len(trajectory) - sequence_length):
            seq_trajectory = trajectory[i:i + sequence_length].values
            # seq_vehicle_type = np.tile(vehicle_type, (sequence_length, 1))
            # seq_combined = np.hstack((seq_trajectory, seq_vehicle_type))
            X.append(seq_trajectory)
            y.append(feasibility)
    return np.array(X), np.array(y)

def reshape_dataset(trajectories, feasibilities, vehicle_type_encoded):
    X, y = [], []

    for trajectory, feasibility, vehicle_type in zip(trajectories, feasibilities, vehicle_type_encoded):
        
        # trajectory_padded = pad_trajectory(trajectory)
        # trajectory_feasibility = pad_trajectory(feasibility)

        # Tile the encoded vehicle type to match the trajectory time steps
        tiled_vehicle_type = np.tile(vehicle_type, (trajectory.shape[0], 1))

        # Concatenate the tiled vehicle type with the trajectory
        trajectory_with_vehicle_type = np.concatenate([trajectory, tiled_vehicle_type], axis=1)
        X.append(trajectory_with_vehicle_type)
        y.append(feasibility)

    return np.array(X), np.array(y)


def reshape_dataset_for_traditional_ml(trajectories, feasibilities, vehicle_type_encoded):
    X, y = None,  None
    # padded_trajectories = list(map(lambda df: pad_trajectory(df), trajectories))
    # padded_feasibilities = list(map(lambda df: pad_feasibility(df), feasibilities))
    
    for trajectory, feasibility, vehicle_type in zip(trajectories, feasibilities, vehicle_type_encoded):

        tiled_vehicle_types = np.tile(vehicle_type, (trajectory.shape[0], 1))
        tiled_trajectory = np.concatenate([trajectory, tiled_vehicle_types], axis=1)   # Concatenate with the encoded vehicle type
        
        if len(feasibility) != len(trajectory):
            print(f"Length mismatch: Feasibility length is {len(feasibility)}, but trajectory length is {len(trajectory)}.")

        X = tiled_trajectory if X is None else np.vstack((X, tiled_trajectory))
        y = feasibility if y is None else np.concatenate([y, feasibility], axis=0)

    X[:, :trajectory.shape[1]] = X[:, :trajectory.shape[1]].astype('float64')
    return np.array(X), np.array(y)
    

In [7]:
def prepare_data_MLSTM_FCN_split():
    
    unique_vehicle_types = np.unique(all_vehicle_types)
    encoder.fit(unique_vehicle_types.reshape(-1, 1))
    encoded_vehicle_types = encoder.transform(np.array(all_vehicle_types).reshape(-1, 1))


    X_train, X_test, y_train, y_test = [], [], [], []

    X, y = reshape_dataset(all_trajectories, all_feasbilities, encoded_vehicle_types)

    X_shuffled = X.copy()
    permutation = np.random.permutation(X.shape[0])
    X_shuffled = X_shuffled[permutation]
    y_shuffled = y[permutation]


    X_train, X_test, y_train, y_test = train_test_split(X_shuffled, y_shuffled, test_size=0.3, random_state=42)
    NUM_FEATURES = np.array(X_train).shape[2]
    TIMESTEPS = max_count

    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train, dtype=int)  # Converts True to 1 and False to 0
    y_test = np.array(y_test, dtype=int)

    X_train = X_train.reshape(X_train.shape[0], X_train.shape[2], X_train.shape[1])
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[2], X_test.shape[1])
    
    return X_train, X_test, y_train, y_test 

In [8]:
from sklearn.model_selection import StratifiedKFold

def prepare_data_MLSTM_FCN_kfold():
    
    unique_vehicle_types = np.unique(all_vehicle_types)
    encoder.fit(unique_vehicle_types.reshape(-1, 1))
    encoded_vehicle_types = encoder.transform(np.array(all_vehicle_types).reshape(-1, 1))
    X, y = reshape_dataset(all_trajectories, all_feasbilities, encoded_vehicle_types)

    # Specify the number of folds (k)
    n_splits = 5  # You can choose the number of folds

    # Initialize StratifiedKFold with the number of splits and random seed (if desired)
    stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Initialize lists to store training and testing indices for each fold
    train_indices_list = []
    test_indices_list = []

    # Split the data into train and test indices for each fold
    for train_indices, test_indices in stratified_kfold.split(X, y):
        train_indices_list.append(train_indices)
        test_indices_list.append(test_indices)

    # Using training and testing indices for cross-validation
    for fold in range(n_splits):
        X_train, X_test = X[train_indices_list[fold]], X[test_indices_list[fold]]
        y_train, y_test = y[train_indices_list[fold]], y[test_indices_list[fold]]
        

    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train, dtype=int)  # Converts True to 1 and False to 0
    y_test = np.array(y_test, dtype=int)

    X_train = X_train.reshape(X_train.shape[0], X_train.shape[2], X_train.shape[1])
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[2], X_test.shape[1])
    
    return X_train, X_test, y_train, y_test


In [9]:
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA


def prepare_data_trad_kfold():
    
    unique_vehicle_types = np.unique(all_vehicle_types)
    encoder.fit(unique_vehicle_types.reshape(-1, 1))
    encoded_vehicle_types = encoder.transform(np.array(all_vehicle_types).reshape(-1, 1))
    X, y = reshape_dataset_for_traditional_ml(all_trajectories, all_feasbilities, encoded_vehicle_types)
  
    # Specify the number of folds (k)
    n_splits = 10  # You can choose the number of folds

    # Initialize StratifiedKFold with the number of splits and random seed (if desired)
    stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Initialize lists to store training and testing indices for each fold
    train_indices_list = []
    test_indices_list = []

    # Split the data into train and test indices for each fold
    for train_indices, test_indices in stratified_kfold.split(X, y):
        train_indices_list.append(train_indices)
        test_indices_list.append(test_indices)

    for fold in range(n_splits):
        X_train, X_test = X[train_indices_list[fold]], X[test_indices_list[fold]]
        y_train, y_test = y[train_indices_list[fold]], y[test_indices_list[fold]]
        
    # Train  model on X_train and y_train, and evaluate it on X_test and y_test for each fold     
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train, dtype=int)  # Converts True to 1 and False to 0
    y_test = np.array(y_test, dtype=int)

    return X_train, X_test, y_train, y_test



def prepare_data_trad_split():
    unique_vehicle_types = np.unique(all_vehicle_types)
    encoder.fit(unique_vehicle_types.reshape(-1, 1))
    encoded_vehicle_types = encoder.transform(np.array(all_vehicle_types).reshape(-1, 1))


    X_train, X_test, y_train, y_test = [], [], [], []

    X, y = reshape_dataset_for_traditional_ml(all_trajectories, all_feasbilities, encoded_vehicle_types)
    # X_shuffled = X.copy()
    # permutation = np.random.permutation(X.shape[0])
    # X_shuffled = X_shuffled[permutation]
    # y_shuffled = y[permutation]
    
    pca = PCA(n_components=10)
    #X = pca.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train, dtype=int)  # Converts True to 1 and False to 0
    y_test = np.array(y_test, dtype=int)
    

    return X_train, X_test, y_train, y_test

    

SVM Classifier

In [10]:
from sklearn import svm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
X_train, X_test, y_train, y_test = prepare_data_trad_split()
print(X_train.shape)
clf = svm.SVC(kernel='rbf')    # or kernel='rbf'
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
precision = precision_score(y_test, y_pred,)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

precision_wgt = precision_score(y_test, y_pred, average='weighted')
recall_wgt = recall_score(y_test, y_pred, average='weighted')
f1_wgt = f1_score(y_test, y_pred, average='weighted')

# Print the weighted results
print(f'Weighted Precision: {precision_wgt}')
print(f'Weighted Recall: {recall_wgt}')
print(f'Weighted F1-Score: {f1_wgt}')


# Print the results
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')
print(f'Accuracy: {accuracy}')


/home/lydiazeleke/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(110347, 7)
Weighted Precision: 0.8199151236049468
Weighted Recall: 0.8201326711857034
Weighted F1-Score: 0.8200212671343249
Precision: 0.737528041875868
Recall: 0.7337655436284408
F1-Score: 0.7356419818859883
Accuracy: 0.8201326711857034


KNN Classifier

In [11]:
# knn
from sklearn.neighbors import KNeighborsClassifier

X_train, X_test, y_train, y_test = prepare_data_trad_kfold()
clf = KNeighborsClassifier(n_neighbors = 6) 
clf.fit(X_train, y_train) 

y_pred = clf.predict(X_test)
precision = precision_score(y_test, y_pred,)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

precision_wgt = precision_score(y_test, y_pred, average='weighted')
recall_wgt = recall_score(y_test, y_pred, average='weighted')
f1_wgt = f1_score(y_test, y_pred, average='weighted')

# Print the weighted results
print(f'Weighted Precision: {precision_wgt}')
print(f'Weighted Recall: {recall_wgt}')
print(f'Weighted F1-Score: {f1_wgt}')

# Print the results
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')
print(f'Accuracy: {accuracy}')

/home/lydiazeleke/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Weighted Precision: 0.8039371758406376
Weighted Recall: 0.8073660552454144
Weighted F1-Score: 0.8042357863611598
Precision: 0.7474820143884892
Recall: 0.6602414742639271
F1-Score: 0.7011584748622203
Accuracy: 0.8073660552454144


Random Forest Classifier

In [12]:
# random forest
from sklearn.ensemble import RandomForestClassifier 

rf_clf = RandomForestClassifier(n_estimators = 150) 
rf_clf.fit(X_train, y_train) 

y_pred = rf_clf.predict(X_test)
precision = precision_score(y_test, y_pred,)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

precision_wgt = precision_score(y_test, y_pred, average='weighted')
recall_wgt = recall_score(y_test, y_pred, average='weighted')
f1_wgt = f1_score(y_test, y_pred, average='weighted')

# Print the weighted results
print(f'Weighted Precision: {precision_wgt}')
print(f'Weighted Recall: {recall_wgt}')
print(f'Weighted F1-Score: {f1_wgt}')


# Print the results
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')
print(f'Accuracy: {accuracy}')

Weighted Precision: 0.97093815028672
Weighted Recall: 0.9709272819546146
Weighted F1-Score: 0.9709324404398404
Precision: 0.9568527918781726
Recall: 0.9582715526371531
F1-Score: 0.9575616467351044
Accuracy: 0.9709272819546146


XGBoost Classifier

In [14]:
# xgboost
import xgboost as xgb 

clf = xgb.XGBClassifier(max_depth = 8, n_estimators = 300)
clf.fit(X_train, y_train) 


y_pred = clf.predict(X_test)
precision = precision_score(y_test, y_pred,)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

precision_wgt = precision_score(y_test, y_pred, average='weighted')
recall_wgt = recall_score(y_test, y_pred, average='weighted')
f1_wgt = f1_score(y_test, y_pred, average='weighted')

# Print the weighted results
print(f'Weighted Precision: {precision_wgt}')
print(f'Weighted Recall: {recall_wgt}')
print(f'Weighted F1-Score: {f1_wgt}')


# Print the results
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')
print(f'Accuracy: {accuracy}')

Weighted Precision: 0.97093815028672
Weighted Recall: 0.9709272819546146
Weighted F1-Score: 0.9709324404398404
Precision: 0.9568527918781726
Recall: 0.9582715526371531
F1-Score: 0.9575616467351044
Accuracy: 0.9709272819546146


MLP Classifier

In [18]:
# fully connected deep neural network
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=(200,200,200), random_state=1)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
accuracy = accuracy_score(y_test, y_pred)

# Print the results
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')
print(f'Accuracy: {accuracy}')

Precision: 0.9494135704647694
Recall: 0.9495396215471616
F1-Score: 0.9492999583209738
Accuracy: 0.9495396215471616


Save the Best Performing Model to Local Directory

In [22]:
# Save the training history
import pickle
import joblib
from joblib import load, dump

path_to_model = os.path.join(current_dir, 'model', 'Feasibility_Prediction_Model')

model_filename = 'random_forest_model.joblib'

# Ensure the directory exists, create if it does not
if not os.path.exists(path_to_model):
    os.makedirs(path_to_model)

# Full path to the model file
model_path = os.path.join(path_to_model, model_filename)

# Save the model
joblib.dump(rf_clf, model_path)



['/home/lydiazeleke/Desktop/encounter_gen_tool/integrated_encounter_tool/LUAS/output/Feasibility_Prediction_Model/random_forest_model.joblib']

In [21]:

# Check if the model file exists
if os.path.exists(model_path):
    # Load the model
    model_loaded = load(model_path)
    print("Model loaded successfully.")
else:
    print("Model file does not exist.")

# Assuming the model loads successfully and you have X_test and X_train ready
if 'model_loaded' in locals():
    # Evaluate on X_test
    y_pred_test = model_loaded.predict(X_test)
    precision_test = precision_score(y_test, y_pred_test, average='weighted')
    recall_test = recall_score(y_test, y_pred_test, average='weighted')
    f1_test = f1_score(y_test, y_pred_test, average='weighted')
    accuracy_test = accuracy_score(y_test, y_pred_test)

    # Print the results
    print(f'Precision: {precision_test}')
    print(f'Recall: {recall_test}')
    print(f'F1-Score: {f1_test}')
    print(f'Accuracy: {accuracy_test}')

Model loaded successfully.
Precision: 0.9716992007277651
Recall: 0.9717258128828796
F1-Score: 0.9717097690408484
Accuracy: 0.9717258128828796
